we take a step back
We will watch at functions of tensorflow, which will help us :)


how you like the final topic project about: how to optimize training with gans?

împort tesn

In [58]:
import tensorflow as tf 

sample_input = tf.random.uniform(shape=(4,2), minval=-3,maxval=3)


In [59]:
#@ is a decorator -> read the documentation 
#add a functionality to a object?
@tf.custom_gradient
def my_sigmoid(x):
    sig_res = 1/(1+tf.exp(-x))
    def grad(upstream):
        diag_of_jacobian = sig_res * (1 - sig_res)
        downstream = upstream*diag_of_jacobian
        return downstream
    return sig_res, grad

print (my_sigmoid(sample_input))

tf.Tensor(
[[0.6738518  0.9387714 ]
 [0.14364387 0.83773047]
 [0.4468105  0.09600986]
 [0.52505213 0.81481093]], shape=(4, 2), dtype=float32)


In [60]:
@tf.custom_gradient
def my_relu(x):
    #relu_res = tf.math.maximum(0,x)
    relu_res = tf.where(x>0,x,tf.zeros_like(x))
    def grad(upstream):
        #downstream = tf.where([x < 0, x > 0]).numpy() * upstream
        d_dx= tf.where(x>0, tf.ones_like(x),tf.zeros_like(x))
        downstream = upstream*d_dx
        return downstream
    return relu_res, grad

print (my_relu(sample_input))

tf.Tensor(
[[0.7256584  2.7299585 ]
 [0.         1.6414375 ]
 [0.         0.        ]
 [0.10029244 1.4815788 ]], shape=(4, 2), dtype=float32)


In [61]:
sample_input = tf.Variable(initial_value=sample_input)
with tf.GradientTape(persistent=True) as tape:
    tf_relu_act = tf.nn.relu(sample_input)
    my_relu_act = my_relu(sample_input)
grads_tf_relu = tape.gradient(tf_relu_act, [sample_input])
grads_own_relu = tape.gradient(my_relu_act, [sample_input])
del tape

print (grads_own_relu, grads_tf_relu)

[<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 1.],
       [0., 1.],
       [0., 0.],
       [1., 1.]], dtype=float32)>] [<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 1.],
       [0., 1.],
       [0., 0.],
       [1., 1.]], dtype=float32)>]


In [64]:
res = []
#tf function will speed up code but it must be tf code (and not python)
sample_input = tf.Variable(initial_value=tf.random.uniform(shape=(10,4,3)))
@tf.function
def some_tf_loop():
    for step in range(10):
        inputs = sample_input[step,:,:]
        with tf.GradientTape(persistent=True) as tape:
            tf_relu_act = tf.nn.relu(sample_input)
        grads_tf_relu = tape.gradient(tf_relu_act, [sample_input])
        res.append(grads_tf_relu)


some_tf_loop()
print (res[0].numpy())

AttributeError: 'list' object has no attribute 'numpy'